In [1]:
# %pip install --upgrade pingouin
%pip install pyvttbl


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io as spio
from scipy.spatial import distance
from IPython.display import display
from statsmodels.stats.anova import AnovaRM
# import pingouin as pg
# import pyvttbl as pt
# OG_DATA_PATH = './TaskPerformance/'
OG_DATA_PATH = './MatData/'
# PLOTS_PATH = './Sabine_poster/map/'
DATA_PATH = './MetaData/'
PROCESSED_DATA_PATH = './csv_files/'
# os.makedirs(os.path.dirname(PLOTS_PATH), exist_ok=True)
os.makedirs(os.path.dirname(DATA_PATH), exist_ok=True)
sns.set(context = "poster", style="white", palette="dark", font_scale=2, rc={'figure.figsize':(12,9)})

In [3]:
# Check valid subjects
subdf = pd.read_csv(DATA_PATH+'./Seahaven_alignment_project.csv', sep=',')

## Getting House angles and coordinates and saving in csv

In [4]:
housedf = pd.read_csv(DATA_PATH + 'complete_list_houses.txt',header=None) 
house_coords = pd.read_csv(DATA_PATH + 'HouseList.txt',header=None)
housedf.columns=['HouseNr_Angle']
house_coords.columns = ['HouseNr_coordinates']
housedf = housedf.HouseNr_Angle.str.split('_', expand=True)
housedf.columns=['HouseNr', 'Angle']
housedf.set_index('HouseNr', inplace=True)
house_coords = house_coords.HouseNr_coordinates.str.split(':', expand=True)
house_coords.columns = ['HouseNr', 'coordinates']
house_coords[['x','y']] = house_coords.coordinates.str.split(';', expand=True)
house_coords.set_index('HouseNr', inplace=True)

housedf = housedf.merge(house_coords, on='HouseNr')
housedf.drop(columns='coordinates', inplace=True)

housedf = housedf.reset_index()
housedf.HouseNr.str.strip()
housedf.Angle = housedf.Angle.str.strip().astype(np.float64)
housedf.x = housedf.x.str.strip().astype(np.float64)
housedf.y = housedf.y.str.strip().astype(np.float64)
# housedf = housedf.astype(np.float64)
display(housedf.info())

housedf.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_house_info.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 4 columns):
HouseNr    193 non-null object
Angle      193 non-null float64
x          193 non-null float64
y          193 non-null float64
dtypes: float64(3), object(1)
memory usage: 6.1+ KB


None

# Finding valid subjects from map and VR groups
* ### Subjects who have finished 3 sessions
* ### Subjects who have not been discarded because of faulty measurement

In [5]:
maptmpdf = (
    subdf
    .query('Discarded != "yes"')
    .query('Measurement == 3')
    .query('Training == "map"')
    [['Subject']] 
)
VRtmpdf = (
    subdf
    .query('Discarded != "yes"')
    .query('Measurement == 3')
    .query('Training == "VR"')
    [['Subject']] 
)
map_val_subs = maptmpdf.Subject.unique()
VR_val_subs = VRtmpdf.Subject.unique()
map_val_subs.sort()
VR_val_subs.sort()
display(map_val_subs)
display(VR_val_subs)

array([1719, 2096, 2294, 2299, 2557, 3426, 4199, 4749, 5048, 5162, 5904,
       6449, 7085, 7396, 7844, 8021, 8078, 8124, 8479, 8802, 9369, 9864],
      dtype=int64)

array([1089, 1155, 2044, 3023, 3743, 4272, 5239, 5253, 5346, 6398, 7021,
       8072, 8195, 8261, 8547, 8699, 8936, 8954, 9017, 9437, 9475, 9961],
      dtype=int64)

# Reading Task performance data from .mat files

In [6]:
def mat_to_df(subNr,taskType):
    '''
    converts mat struct with task results into pandas df
    also adds extra column with information whether trial was correct or wrong    
    '''
    columns = {'Absolute': ['HouseNr', 'Correct_Angle', 'Wrong_Angle', 'Correct_Answer','RT',
               'Sub_Answer', 'Accurate', 'Time', 'Task', 'Subject'],
               
               'Relative': ['PrimeNr', 'TargetNr_correct', 'TargetNr_wrong', 'Correct_Answer','RT',
               'Sub_Answer', 'Accurate', 'Time', 'Task', 'Subject'],
               
               'Pointing': ['PrimeNr', 'TargetNr', 'Angle_correct', 'Angle_wrong','Correct_Answer','RT',
               'Sub_Answer', 'Accurate', 'Time', 'Task', 'Subject']
              }
    matpath = OG_DATA_PATH+"/AlignmentVR_SubjNo_"+ str(subNr) +".mat"
    if os.path.exists(matpath):
        mat_contents = spio.loadmat(matpath)
        type_array = []
        for j,cond_2 in enumerate(["Trial_3s", "Trial_Inf"]):
            trials_array = []
            for line in range(len(mat_contents['Output'][0][0][taskType][cond_2][0][0])):
                value_array = []
                for column in range(len(mat_contents['Output'][0][0][taskType][cond_2][0][0][line][0])):
                    value = mat_contents['Output'][0][0][taskType][cond_2][0][0][line][0][column][0][0]
                    value_array.append(value)
                # check if trial is correct(true or false
                value_array.append(value_array[-1] == value_array[-3])
                value_array.extend([cond_2.split('_')[1],taskType, subNr]) 
    #                 print(value_array)
                trials_array.append(value_array)
    #             print(len(value_array))
            type_array.extend(trials_array)
    #     print(type_array)
        return pd.DataFrame.from_records(type_array, columns=columns[taskType])
    else:
        print(matpath+' NOT FOUND!')
        return pd.DataFrame()

# Reading all Map data

In [7]:
abs_df = pd.concat([mat_to_df(sub, 'Absolute') for idx, sub in enumerate(map_val_subs)],
            ignore_index=True) 
display(abs_df.head())
abs_df['AngularDiff'] = abs(abs_df.Correct_Angle - abs_df.Wrong_Angle)
abs_df.loc[abs_df['AngularDiff']>180, 'AngularDiff' ] = 360 - abs_df['AngularDiff']
abs_df['AngularDiff'] = abs_df['AngularDiff'].astype(str)
abs_df['Correct_Angle_180'] = abs_df.Correct_Angle
abs_df.loc[abs_df['Correct_Angle_180']>180, 'Correct_Angle_180' ] = 360 - abs_df['Correct_Angle_180']
# display(abs_df.Correct_Angle_180.value_counts(bins=7))
# bins = pd.IntervalIndex.from_tuples([(0, 10), (15, 45), (45, 75),(75,105),(105,135),(135,165),(165,195)])
bins = [0,15,45,75,105,135,165,195]
abs_df['AngleToNorth'], bins = pd.cut(abs_df.Correct_Angle_180, bins=7,labels=[0,30,60,90,120,150,180], retbins=True)
abs_df['AngleToNorth'] = abs_df['AngleToNorth'].astype(str)
display(abs_df.AngleToNorth.value_counts())

,HouseNr,Correct_Angle,Wrong_Angle,Correct_Answer,RT,Sub_Answer,Accurate,Time,Task,Subject
0,170,243,93,U,1.911848,D,False,3s,Absolute,1719
1,140,303,243,U,1.730624,U,True,3s,Absolute,1719
2,177,155,305,D,1.968733,D,True,3s,Absolute,1719
3,132,120,330,U,2.527483,D,False,3s,Absolute,1719
4,122,30,60,D,2.286321,D,True,3s,Absolute,1719


150    286
60     264
90     264
120    242
30     242
180    154
0      132
Name: AngleToNorth, dtype: int64

In [8]:
rel_df = pd.concat([mat_to_df(sub, 'Relative') for idx, sub in enumerate(map_val_subs)],
            ignore_index=True) 
housedf.HouseNr = housedf.HouseNr.astype(int)

rel_df['Correct_Angle'] = rel_df.merge(housedf, left_on='TargetNr_correct', right_on='HouseNr', how='left')['Angle']
rel_df['Wrong_Angle'] = rel_df.merge(housedf, left_on='TargetNr_wrong', right_on='HouseNr', how='left')['Angle']
rel_df[['target_x', 'target_y']] = rel_df.merge(housedf, left_on='TargetNr_correct', right_on='HouseNr', how='left')[['x','y']]
rel_df[['prime_x', 'prime_y']] = rel_df.merge(housedf, left_on='PrimeNr', right_on='HouseNr', how='left')[['x','y']]
rel_df['distance_prime_target'] = np.sqrt((rel_df.prime_x - rel_df.target_x)**2 + (rel_df.prime_y - rel_df.target_y)**2 )
rel_df['AngularDiff'] = abs(rel_df.Correct_Angle - rel_df.Wrong_Angle)
rel_df.loc[rel_df['AngularDiff']>180, 'AngularDiff' ] = 360 - rel_df['AngularDiff']

# bins = pd.IntervalIndex.from_tuples([(0, 45), (45, 75),(75,105),(105,135),(135,165),(165,195)])
bins = [0,45,75,105,135,165,195]
rel_df['AngularDiff'] = pd.cut(rel_df.AngularDiff, bins=bins,labels=[30,60,90,120,150,180], retbins=False)

rel_df['AngularDiff'] = rel_df['AngularDiff'].astype(str)

display(rel_df.AngularDiff.value_counts())
display(rel_df.head())


180    264
60     264
150    264
120    264
30     264
90     264
Name: AngularDiff, dtype: int64

,PrimeNr,TargetNr_correct,TargetNr_wrong,Correct_Answer,RT,Sub_Answer,Accurate,Time,Task,Subject,Correct_Angle,Wrong_Angle,target_x,target_y,prime_x,prime_y,distance_prime_target,AngularDiff
0,171,147,88,D,2.398472,D,True,3s,Relative,1719,63.0,90.0,426.5,754.8,316.8,824.3,129.862774,30
1,189,190,19,U,2.487582,U,True,3s,Relative,1719,60.0,180.0,230.3,834.0,213.6,866.2,36.272993,120
2,117,110,53,U,2.156479,D,False,3s,Relative,1719,30.0,90.0,480.1,813.0,394.9,790.9,88.019600,60
3,180,169,125,D,1.408578,D,True,3s,Relative,1719,325.0,125.0,349.0,813.9,522.0,830.2,173.766193,150
4,55,50,85,D,1.669780,U,False,3s,Relative,1719,90.0,270.0,431.4,740.6,437.6,803.8,63.503386,180


In [9]:
point_df = pd.concat([mat_to_df(sub, 'Pointing') for idx, sub in enumerate(map_val_subs)],
            ignore_index=True) 
point_df['AngularDiff'] = abs(point_df.Angle_correct - point_df.Angle_wrong)
point_df.loc[point_df['AngularDiff']>180, 'AngularDiff' ] = 360 - point_df['AngularDiff']
point_df['AngularDiff'] = point_df['AngularDiff'].astype(str)
point_df[['target_x', 'target_y']] = point_df.merge(housedf, left_on='TargetNr', right_on='HouseNr', how='left')[['x','y']]
point_df[['prime_x', 'prime_y']] = point_df.merge(housedf, left_on='PrimeNr', right_on='HouseNr', how='left')[['x','y']]
point_df['distance_prime_target'] = np.sqrt((point_df.prime_x - point_df.target_x)**2 + 
                                            (point_df.prime_y - point_df.target_y)**2 )

display(point_df.head())



,PrimeNr,TargetNr,Angle_correct,Angle_wrong,Correct_Answer,RT,Sub_Answer,Accurate,Time,Task,Subject,AngularDiff,target_x,target_y,prime_x,prime_y,distance_prime_target
0,33,9,243,213,D,2.061093,D,True,3s,Pointing,1719,30,541.8,795.2,495.2,885.6,101.704081
1,119,146,132,72,U,NaN,N,False,3s,Pointing,1719,60,382.4,891.4,412.0,779.9,115.362082
2,156,67,306,126,D,1.653943,D,True,3s,Pointing,1719,180,290.0,783.1,462.5,656.6,213.912365
3,81,37,88,208,U,1.437038,D,False,3s,Pointing,1719,120,442.3,889.0,439.8,950.2,61.251041
4,133,65,226,16,U,1.886768,D,False,3s,Pointing,1719,150,437.4,671.2,597.1,827.9,223.738642


In [10]:

# Make empty dataframe with task,SubjectID, PrimeNr, TargetNr , Accuracy
#save this house_task as csv
house_task_df = pd.DataFrame()

house_task_df = pd.concat([house_task_df, abs_df.loc[:,['Subject','Task','HouseNr',
                                                    'Time','Accurate']]], ignore_index=True)
house_task_df = pd.concat([house_task_df, rel_df.loc[:,['Subject','Task','PrimeNr','TargetNr_correct','TargetNr_wrong',
                                                'Time','Accurate']]], ignore_index=True)
house_task_df = pd.concat([house_task_df, point_df.loc[:,['Subject','Task','PrimeNr','TargetNr',
                                                'Time','Accurate']]], ignore_index=True)
house_task_df['Group'] = 'Map'
house_task_df = house_task_df.fillna('')
house_task_df[['HouseNr','PrimeNr','TargetNr','TargetNr_correct','TargetNr_wrong']] = house_task_df[['HouseNr',\
                                                 'PrimeNr','TargetNr','TargetNr_correct','TargetNr_wrong']].astype(str)


house_task_df['HouseNr'] = house_task_df['HouseNr'] + house_task_df['PrimeNr']
house_task_df['TargetNr'] = house_task_df['TargetNr'] + house_task_df['TargetNr_correct']
house_task_df.drop(columns=['PrimeNr','TargetNr_correct'], inplace=True)
house_task_df.rename(columns={'Accurate':'Accuracy','HouseNr':'PrimeNr'}, inplace=True)
display(house_task_df)
house_task_df.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_map_house_task_accuracy.csv', index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


,Accuracy,PrimeNr,Subject,TargetNr,TargetNr_wrong,Task,Time,Group
0,False,170.0,1719,,,Absolute,3s,Map
1,True,140.0,1719,,,Absolute,3s,Map
2,True,177.0,1719,,,Absolute,3s,Map
3,False,132.0,1719,,,Absolute,3s,Map
4,True,122.0,1719,,,Absolute,3s,Map
5,False,194.0,1719,,,Absolute,3s,Map
6,False,58.0,1719,,,Absolute,3s,Map
7,True,147.0,1719,,,Absolute,3s,Map
8,False,196.0,1719,,,Absolute,3s,Map
9,True,17.0,1719,,,Absolute,3s,Map


In [11]:
north_alignment_map = (
    abs_df
    .groupby(['Subject','AngleToNorth','Time'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
)
north_alignment_map['Group'] = 'map'
display(north_alignment_map)

df = pd.pivot_table(north_alignment_map, index=['Subject','Group'], values='Accuracy',columns=['AngleToNorth','Time'],
                    aggfunc='mean')
display(df.columns)
df.columns =['_'.join(col).rstrip('_') for col in df.columns.values]
display(df.columns)
df.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_map_angletonorth_accuracy_time_wideform.csv', index=False)

,Subject,AngleToNorth,Time,Accuracy,Group
0,1719,0,3s,0.750000,map
1,1719,0,Inf,0.500000,map
2,1719,120,3s,0.250000,map
3,1719,120,Inf,0.428571,map
4,1719,150,3s,0.000000,map
5,1719,150,Inf,0.571429,map
6,1719,180,3s,0.666667,map
7,1719,180,Inf,0.750000,map
8,1719,30,3s,0.833333,map
9,1719,30,Inf,1.000000,map


MultiIndex(levels=[['0', '120', '150', '180', '30', '60', '90'], ['3s', 'Inf']],
           codes=[[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]],
           names=['AngleToNorth', 'Time'])

Index(['0_3s', '0_Inf', '120_3s', '120_Inf', '150_3s', '150_Inf', '180_3s',
       '180_Inf', '30_3s', '30_Inf', '60_3s', '60_Inf', '90_3s', '90_Inf'],
      dtype='object')

In [12]:
map_perf_df = pd.concat([
    (
    abs_df
    .groupby(['Subject','Task','Time'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    ),
    (
    rel_df
    .groupby(['Subject','Task','Time'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    ),
    (
    point_df
    .groupby(['Subject','Task','Time'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    )
]
)
map_perf_df['Group'] = 'Map'
display(map_perf_df.head())

,Subject,Task,Time,Accuracy,Group
0,1719,Absolute,3s,0.500000,Map
1,1719,Absolute,Inf,0.611111,Map
2,2096,Absolute,3s,0.527778,Map
3,2096,Absolute,Inf,0.611111,Map
4,2294,Absolute,3s,0.555556,Map


In [13]:
map_angdiff_df = pd.concat([
    (
    abs_df
    .groupby(['Subject','Task','Time','AngularDiff'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    ),
    (
    rel_df
    .groupby(['Subject','Task','Time','AngularDiff'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    ),
    (
    point_df
    .groupby(['Subject','Task','Time','AngularDiff'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    )
]
)
map_angdiff_df['Group'] = 'map'
display(map_angdiff_df)

,Subject,Task,Time,AngularDiff,Accuracy,Group
0,1719,Absolute,3s,120,0.666667,map
1,1719,Absolute,3s,150,0.166667,map
2,1719,Absolute,3s,180,0.666667,map
3,1719,Absolute,3s,30,0.666667,map
4,1719,Absolute,3s,60,0.500000,map
5,1719,Absolute,3s,90,0.333333,map
6,1719,Absolute,Inf,120,0.500000,map
7,1719,Absolute,Inf,150,0.666667,map
8,1719,Absolute,Inf,180,0.500000,map
9,1719,Absolute,Inf,30,0.666667,map


In [14]:
tmpdf = pd.concat([
    (
    rel_df
    .groupby(['Task','Time','PrimeNr','TargetNr_correct', 'distance_prime_target'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    ),
    (
    point_df
    .groupby(['Task','Time','PrimeNr','TargetNr','distance_prime_target'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    )
    
])
tmpdf['Group'] = 'Map'
tmpdf.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_map_rel_point_distance_accuracy.csv', index=False)

display(tmpdf)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


,Accuracy,PrimeNr,TargetNr,TargetNr_correct,Task,Time,distance_prime_target,Group
0,0.681818,15,NaN,18.0,Relative,3s,31.740353,Map
1,0.772727,15,NaN,36.0,Relative,3s,158.913089,Map
2,0.681818,33,NaN,38.0,Relative,3s,60.406705,Map
3,0.772727,33,NaN,73.0,Relative,3s,69.766539,Map
4,0.681818,39,NaN,40.0,Relative,3s,91.731401,Map
5,0.590909,39,NaN,83.0,Relative,3s,37.251846,Map
6,0.681818,55,NaN,50.0,Relative,3s,63.503386,Map
7,0.590909,55,NaN,59.0,Relative,3s,90.921725,Map
8,0.500000,76,NaN,72.0,Relative,3s,106.977614,Map
9,0.636364,76,NaN,99.0,Relative,3s,130.100961,Map


# Reading all VR data

In [15]:
abs_df = pd.concat([mat_to_df(sub, 'Absolute') for idx, sub in enumerate(VR_val_subs)],
            ignore_index=True) 
abs_df['AngularDiff'] = abs(abs_df.Correct_Angle - abs_df.Wrong_Angle)
abs_df.loc[abs_df['AngularDiff']>180, 'AngularDiff' ] = 360 - abs_df['AngularDiff']
abs_df['AngularDiff'] = abs_df['AngularDiff'].astype(str)
abs_df['Correct_Angle_180'] = abs_df.Correct_Angle
abs_df.loc[abs_df['Correct_Angle_180']>180, 'Correct_Angle_180' ] = 360 - abs_df['Correct_Angle_180']
# display(abs_df.Correct_Angle_180.value_counts(bins=7))
# bins = pd.IntervalIndex.from_tuples([(0, 10), (15, 45), (45, 75),(75,105),(105,135),(135,165),(165,195)])
bins = [0,15,45,75,105,135,165,195]
abs_df['AngleToNorth'], bins = pd.cut(abs_df.Correct_Angle_180, bins=7,labels=[0,30,60,90,120,150,180], retbins=True)
abs_df['AngleToNorth'] = abs_df['AngleToNorth'].astype(str)
display(abs_df.AngleToNorth.value_counts())

150    286
60     264
90     264
120    242
30     242
180    154
0      132
Name: AngleToNorth, dtype: int64

In [16]:
rel_df = pd.concat([mat_to_df(sub, 'Relative') for idx, sub in enumerate(VR_val_subs)],
            ignore_index=True) 

rel_df['Correct_Angle'] = rel_df.merge(housedf, left_on='TargetNr_correct', right_on='HouseNr', how='left')['Angle']
rel_df['Wrong_Angle'] = rel_df.merge(housedf, left_on='TargetNr_wrong', right_on='HouseNr', how='left')['Angle']
rel_df[['target_x', 'target_y']] = rel_df.merge(housedf, left_on='TargetNr_correct', right_on='HouseNr', how='left')[['x','y']]
rel_df[['prime_x', 'prime_y']] = rel_df.merge(housedf, left_on='PrimeNr', right_on='HouseNr', how='left')[['x','y']]
rel_df['distance_prime_target'] = np.sqrt((rel_df.prime_x - rel_df.target_x)**2 + (rel_df.prime_y - rel_df.target_y)**2 )
rel_df['AngularDiff'] = abs(rel_df.Correct_Angle - rel_df.Wrong_Angle)
rel_df.loc[rel_df['AngularDiff']>180, 'AngularDiff' ] = 360 - rel_df['AngularDiff']

# bins = pd.IntervalIndex.from_tuples([(0, 45), (45, 75),(75,105),(105,135),(135,165),(165,195)])
bins = [0,45,75,105,135,165,195]
rel_df['AngularDiff'] = pd.cut(rel_df.AngularDiff, bins=bins,labels=[30,60,90,120,150,180], retbins=False)

rel_df['AngularDiff'] = rel_df['AngularDiff'].astype(str)

display(rel_df.AngularDiff.value_counts())
display(rel_df.head())

180    264
60     264
150    264
120    264
30     264
90     264
Name: AngularDiff, dtype: int64

,PrimeNr,TargetNr_correct,TargetNr_wrong,Correct_Answer,RT,Sub_Answer,Accurate,Time,Task,Subject,Correct_Angle,Wrong_Angle,target_x,target_y,prime_x,prime_y,distance_prime_target,AngularDiff
0,33,38,178,D,0.702042,D,True,3s,Relative,1089,0.0,60.0,439.1,863.2,495.2,885.6,60.406705,60
1,171,147,88,U,1.466819,U,True,3s,Relative,1089,63.0,90.0,426.5,754.8,316.8,824.3,129.862774,30
2,15,18,137,U,1.140642,U,True,3s,Relative,1089,0.0,120.0,399.5,717.9,397.9,749.6,31.740353,120
3,15,36,195,U,1.086501,U,True,3s,Relative,1089,0.0,148.0,437.5,903.5,397.9,749.6,158.913089,150
4,189,191,67,D,0.956194,D,True,3s,Relative,1089,60.0,270.0,222.6,827.6,213.6,866.2,39.635338,150


In [17]:
point_df = pd.concat([mat_to_df(sub, 'Pointing') for idx, sub in enumerate(VR_val_subs)],
            ignore_index=True) 
point_df['AngularDiff'] = abs(point_df.Angle_correct - point_df.Angle_wrong)
point_df.loc[point_df['AngularDiff']>180, 'AngularDiff' ] = 360 - point_df['AngularDiff']
point_df['AngularDiff'] = point_df['AngularDiff'].astype(str)
point_df[['target_x', 'target_y']] = point_df.merge(housedf, left_on='TargetNr', right_on='HouseNr', how='left')[['x','y']]
point_df[['prime_x', 'prime_y']] = point_df.merge(housedf, left_on='PrimeNr', right_on='HouseNr', how='left')[['x','y']]
point_df['distance_prime_target'] = np.sqrt((point_df.prime_x - point_df.target_x)**2 + 
                                            (point_df.prime_y - point_df.target_y)**2 )

display(point_df.head())

,PrimeNr,TargetNr,Angle_correct,Angle_wrong,Correct_Answer,RT,Sub_Answer,Accurate,Time,Task,Subject,AngularDiff,target_x,target_y,prime_x,prime_y,distance_prime_target
0,180,72,273,93,U,1.785180,U,True,3s,Pointing,1089,180,284.2,841.5,522.0,830.2,238.068331
1,39,30,164,134,U,1.813846,D,False,3s,Pointing,1089,30,361.1,794.5,647.9,876.3,298.237288
2,193,176,228,48,D,1.684182,U,False,3s,Pointing,1089,180,598.2,792.3,345.3,872.1,265.191346
3,55,111,147,27,D,1.847035,D,True,3s,Pointing,1089,120,416.0,704.8,437.6,803.8,101.328969
4,81,37,88,208,D,2.140585,D,True,3s,Pointing,1089,120,442.3,889.0,439.8,950.2,61.251041


In [18]:
# Make empty dataframe with task,SubjectID, PrimeNr, TargetNr , Accuracy
#save this house_task as csv
house_task_df = pd.DataFrame()

house_task_df = pd.concat([house_task_df, abs_df.loc[:,['Subject','Task','HouseNr',
                                                    'Time','Accurate']]], ignore_index=True)
house_task_df = pd.concat([house_task_df, rel_df.loc[:,['Subject','Task','PrimeNr','TargetNr_correct','TargetNr_wrong',
                                                'Time','Accurate']]], ignore_index=True)
house_task_df = pd.concat([house_task_df, point_df.loc[:,['Subject','Task','PrimeNr','TargetNr',
                                                'Time','Accurate']]], ignore_index=True)
house_task_df['Group'] = 'VR'
house_task_df = house_task_df.fillna('')
house_task_df[['HouseNr','PrimeNr','TargetNr','TargetNr_correct','TargetNr_wrong']] = house_task_df[['HouseNr',\
                                                 'PrimeNr','TargetNr','TargetNr_correct','TargetNr_wrong']].astype(str)


house_task_df['HouseNr'] = house_task_df['HouseNr'] + house_task_df['PrimeNr']
house_task_df['TargetNr'] = house_task_df['TargetNr'] + house_task_df['TargetNr_correct']
house_task_df.drop(columns=['PrimeNr','TargetNr_correct'], inplace=True)
house_task_df.rename(columns={'Accurate':'Accuracy','HouseNr':'PrimeNr'}, inplace=True)
display(house_task_df)
house_task_df.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_VR_house_task_accuracy.csv', index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


,Accuracy,PrimeNr,Subject,TargetNr,TargetNr_wrong,Task,Time,Group
0,True,55.0,1089,,,Absolute,3s,VR
1,False,112.0,1089,,,Absolute,3s,VR
2,False,70.0,1089,,,Absolute,3s,VR
3,True,134.0,1089,,,Absolute,3s,VR
4,False,194.0,1089,,,Absolute,3s,VR
5,True,171.0,1089,,,Absolute,3s,VR
6,False,168.0,1089,,,Absolute,3s,VR
7,True,176.0,1089,,,Absolute,3s,VR
8,False,145.0,1089,,,Absolute,3s,VR
9,False,132.0,1089,,,Absolute,3s,VR


In [19]:
north_alignment_VR = (
    abs_df
    .groupby(['Subject','AngleToNorth','Time'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
)
north_alignment_VR['Group'] = 'VR'
display(north_alignment_VR)
df = pd.pivot_table(north_alignment_VR, index=['Subject','Group'], values='Accuracy',columns=['AngleToNorth','Time'],
                    aggfunc='mean').reset_index()
display(df.head())
df.columns =['_'.join(col).rstrip('_') for col in df.columns.values]
display(df.columns)
df.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_VR_angletonorth_accuracy_time_wideform.csv', index=False)

,Subject,AngleToNorth,Time,Accuracy,Group
0,1089,0,3s,0.250000,VR
1,1089,0,Inf,0.000000,VR
2,1089,120,3s,0.500000,VR
3,1089,120,Inf,0.142857,VR
4,1089,150,3s,0.500000,VR
5,1089,150,Inf,0.571429,VR
6,1089,180,3s,0.333333,VR
7,1089,180,Inf,0.500000,VR
8,1089,30,3s,0.333333,VR
9,1089,30,Inf,0.600000,VR


AngleToNorth Subject Group     0        120                 150            \
Time                          3s  Inf    3s       Inf        3s       Inf   
0               1089    VR  0.25  0.0  0.50  0.142857  0.500000  0.571429   
1               1155    VR  0.25  0.0  0.25  0.285714  0.333333  0.714286   
2               2044    VR  0.50  0.0  0.75  0.428571  0.500000  0.142857   
3               3023    VR  0.50  1.0  1.00  0.571429  0.333333  1.000000   
4               3743    VR  0.75  1.0  0.50  0.714286  0.166667  0.857143   

AngleToNorth       180              30             60             90            
Time                3s   Inf        3s  Inf        3s  Inf        3s       Inf  
0             0.333333  0.50  0.333333  0.6  0.285714  0.6  0.666667  0.666667  
1             0.333333  0.50  0.666667  0.6  0.428571  0.6  0.833333  0.500000  
2             0.333333  0.25  0.500000  0.6  0.428571  0.6  0.166667  0.666667  
3             1.000000  0.50  1.000000  0.8  0.571429  0.6  0.500000  0.833333  
4             0.333333  1.00  0.500000  0.8  0.285714  0.4  0.666667  0.833333

Index(['Subject', 'Group', '0_3s', '0_Inf', '120_3s', '120_Inf', '150_3s',
       '150_Inf', '180_3s', '180_Inf', '30_3s', '30_Inf', '60_3s', '60_Inf',
       '90_3s', '90_Inf'],
      dtype='object')

In [20]:
VR_perf_df = pd.concat([
    (
    abs_df
    .groupby(['Subject','Task','Time'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    ),
    (
    rel_df
    .groupby(['Subject','Task','Time'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    ),
    (
    point_df
    .groupby(['Subject','Task','Time'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    )
]
)
VR_perf_df['Group'] = 'VR'
# display(VR_perf_df.isna().sum())
display(VR_perf_df)

,Subject,Task,Time,Accuracy,Group
0,1089,Absolute,3s,0.416667,VR
1,1089,Absolute,Inf,0.472222,VR
2,1155,Absolute,3s,0.472222,VR
3,1155,Absolute,Inf,0.500000,VR
4,2044,Absolute,3s,0.444444,VR
5,2044,Absolute,Inf,0.416667,VR
6,3023,Absolute,3s,0.666667,VR
7,3023,Absolute,Inf,0.750000,VR
8,3743,Absolute,3s,0.444444,VR
9,3743,Absolute,Inf,0.777778,VR


In [21]:
all_data = pd.concat([map_perf_df,VR_perf_df])
display(all_data.isna().sum())
all_data.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_group_task_time_accuracy.csv', index=False)

# pd.melt(all_data,id_vars=['Subject','Group'],value_vars=[])
df = pd.pivot_table(all_data, index=['Subject','Group'], values='Accuracy',columns=['Task','Time']).reset_index()
display(df.isna().sum())
df.columns =['_'.join(col).rstrip('_') for col in df.columns.values]
display(df.isna().sum())
df.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_group_task_time_accuracy_wideform.csv', index=False)

Subject     0
Task        0
Time        0
Accuracy    0
Group       0
dtype: int64

Task      Time
Subject           0
Group             0
Absolute  3s      0
          Inf     0
Pointing  3s      0
          Inf     0
Relative  3s      0
          Inf     0
dtype: int64

Subject         0
Group           0
Absolute_3s     0
Absolute_Inf    0
Pointing_3s     0
Pointing_Inf    0
Relative_3s     0
Relative_Inf    0
dtype: int64

In [22]:
VR_angdiff_df = pd.concat([
    (
    abs_df
    .groupby(['Subject','Task','Time','AngularDiff'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    ),
    (
    rel_df
    .groupby(['Subject','Task','Time','AngularDiff'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    ),
    (
    point_df
    .groupby(['Subject','Task','Time','AngularDiff'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    )
]
)
VR_angdiff_df['Group'] = 'VR'
display(VR_angdiff_df)

,Subject,Task,Time,AngularDiff,Accuracy,Group
0,1089,Absolute,3s,120,0.500000,VR
1,1089,Absolute,3s,150,0.166667,VR
2,1089,Absolute,3s,180,0.166667,VR
3,1089,Absolute,3s,30,0.833333,VR
4,1089,Absolute,3s,60,0.500000,VR
5,1089,Absolute,3s,90,0.333333,VR
6,1089,Absolute,Inf,120,0.666667,VR
7,1089,Absolute,Inf,150,0.666667,VR
8,1089,Absolute,Inf,180,0.333333,VR
9,1089,Absolute,Inf,30,0.666667,VR


In [23]:
tmpdf = pd.concat([
    (
    rel_df
    .groupby(['Task','Time','PrimeNr','TargetNr_correct', 'distance_prime_target'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    ),
    (
    point_df
    .groupby(['Task','Time','PrimeNr','TargetNr','distance_prime_target'])
    .agg({'Accurate':'mean'})
    .rename(columns={'Accurate':'Accuracy'})
    .reset_index()
    )
    
])
tmpdf['Group'] = 'VR'
tmpdf.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_VR_rel_point_distance_accuracy.csv', index=False)

display(tmpdf)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


,Accuracy,PrimeNr,TargetNr,TargetNr_correct,Task,Time,distance_prime_target,Group
0,0.454545,15,NaN,18.0,Relative,3s,31.740353,VR
1,0.590909,15,NaN,36.0,Relative,3s,158.913089,VR
2,0.545455,33,NaN,38.0,Relative,3s,60.406705,VR
3,0.681818,33,NaN,73.0,Relative,3s,69.766539,VR
4,0.454545,39,NaN,40.0,Relative,3s,91.731401,VR
5,0.681818,39,NaN,83.0,Relative,3s,37.251846,VR
6,0.681818,55,NaN,50.0,Relative,3s,63.503386,VR
7,0.454545,55,NaN,59.0,Relative,3s,90.921725,VR
8,0.409091,76,NaN,72.0,Relative,3s,106.977614,VR
9,0.636364,76,NaN,99.0,Relative,3s,130.100961,VR


In [24]:
all_angdiff_data = pd.concat([map_angdiff_df,VR_angdiff_df])
display(all_angdiff_data)
all_angdiff_data.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_group_angulardifference_time_accuracy.csv', index=False)
df = pd.pivot_table(all_angdiff_data, index=['Subject','Group'], values='Accuracy',columns=['AngularDiff','Time'],
                    aggfunc='mean').reset_index()
display(df.columns)
df.columns =['_'.join(col).rstrip('_') for col in df.columns.values]
display(df.columns)
df.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_group_angulardifference_time_accuracy_wideform.csv', index=False)

,Subject,Task,Time,AngularDiff,Accuracy,Group
0,1719,Absolute,3s,120,0.666667,map
1,1719,Absolute,3s,150,0.166667,map
2,1719,Absolute,3s,180,0.666667,map
3,1719,Absolute,3s,30,0.666667,map
4,1719,Absolute,3s,60,0.500000,map
5,1719,Absolute,3s,90,0.333333,map
6,1719,Absolute,Inf,120,0.500000,map
7,1719,Absolute,Inf,150,0.666667,map
8,1719,Absolute,Inf,180,0.500000,map
9,1719,Absolute,Inf,30,0.666667,map


MultiIndex(levels=[['120', '150', '180', '30', '60', '90', 'Group', 'Subject'], ['3s', 'Inf', '']],
           codes=[[7, 6, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5], [2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]],
           names=['AngularDiff', 'Time'])

Index(['Subject', 'Group', '120_3s', '120_Inf', '150_3s', '150_Inf', '180_3s',
       '180_Inf', '30_3s', '30_Inf', '60_3s', '60_Inf', '90_3s', '90_Inf'],
      dtype='object')

In [25]:
north_alignment = pd.concat([north_alignment_map,north_alignment_VR])
display(north_alignment)
north_alignment.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_group_angletonorth_time_accuracy.csv', index=False)

df = pd.pivot_table(north_alignment, index=['Group','Subject'], values='Accuracy',columns=['AngleToNorth','Time'],
                    aggfunc='mean').reset_index()
display(df.columns)
df.columns =['_'.join(col).rstrip('_') for col in df.columns.values]
display(df.columns)
df.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_group_angletonorth_time_accuracy_wideform.csv', index=False)

,Subject,AngleToNorth,Time,Accuracy,Group
0,1719,0,3s,0.750000,map
1,1719,0,Inf,0.500000,map
2,1719,120,3s,0.250000,map
3,1719,120,Inf,0.428571,map
4,1719,150,3s,0.000000,map
5,1719,150,Inf,0.571429,map
6,1719,180,3s,0.666667,map
7,1719,180,Inf,0.750000,map
8,1719,30,3s,0.833333,map
9,1719,30,Inf,1.000000,map


MultiIndex(levels=[['0', '120', '150', '180', '30', '60', '90', 'Subject', 'Group'], ['3s', 'Inf', '']],
           codes=[[8, 7, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6], [2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]],
           names=['AngleToNorth', 'Time'])

Index(['Group', 'Subject', '0_3s', '0_Inf', '120_3s', '120_Inf', '150_3s',
       '150_Inf', '180_3s', '180_Inf', '30_3s', '30_Inf', '60_3s', '60_Inf',
       '90_3s', '90_Inf'],
      dtype='object')

In [26]:
rel_df.columns

Index(['PrimeNr', 'TargetNr_correct', 'TargetNr_wrong', 'Correct_Answer', 'RT',
       'Sub_Answer', 'Accurate', 'Time', 'Task', 'Subject', 'Correct_Angle',
       'Wrong_Angle', 'target_x', 'target_y', 'prime_x', 'prime_y',
       'distance_prime_target', 'AngularDiff'],
      dtype='object')

# Get subject numbers for all 3 measurements

### Get subject indices and merge with FRS scales to join map task accuracy and FRS score in one df

In [45]:
FRS_map = pd.read_csv(DATA_PATH + 'FRS_scales_Map.csv', sep=';', na_values=' ') 
display(FRS_map.info())
FRS_map.columns = ['VP', 'Gender','Age', 'Egocentric','Allocentric','Cardinal Direction']
FRS_map.VP = FRS_map.VP.astype(str)
FRS_map_long = pd.melt(FRS_map, id_vars=['VP'], value_vars=['Egocentric','Allocentric','Cardinal Direction'],
                     var_name='Scale',value_name='Score')
display(FRS_map.info())
tmpdf = (
    subdf
    .query('Discarded != "yes"')
    .query('Measurement == 3')
    .query('Training == "map"')
    [['Subject','Comments']]
)
tmpdf.Subject = tmpdf.Subject.astype(str)
tmpdf = tmpdf.set_index('Subject')

display(tmpdf.info())
tmpdf[['measurement_1','measurement_2']] = tmpdf.Comments.str.replace(r'[A-Za-z\s\-\&\.\']','')\
                                                .str.strip()\
                                                .str.extract('\#([0-9]*)\#([0-9]*)', expand = True)
display(tmpdf)
tmpdf.drop(columns='Comments', inplace=True)
tmpdf.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_map_repeated_measurement_subID.csv')

map_perf_df.Subject = map_perf_df.Subject.astype(str)
map_measurements = map_perf_df.merge(tmpdf, on='Subject')
# map_measurements.
map_FRS_long = map_measurements.merge(FRS_map_long,left_on='measurement_1', right_on='VP')
display(map_FRS_long)
map_FRS_long.drop(columns=['VP'], inplace=True)
map_FRS_long.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_map_task_time_FRS_long.csv', index=False)

pivot_map = pd.pivot_table(map_perf_df, 
                   index=['Subject','Group'], 
                   values='Accuracy',
                   columns=['Task','Time'],
                   aggfunc='mean').reset_index()
pivot_map.Subject = pivot_map.Subject.astype(str)
pivot_map.columns = ['_'.join(col).rstrip('_') for col in pivot_map.columns.values]
tmpdf = tmpdf.merge(pivot_map, on='Subject')

tmpdf = tmpdf.merge(FRS_map, left_on='measurement_1', right_on='VP')
tmpdf.drop(columns=['VP'], inplace=True)
tmpdf.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_map_task_time_FRS_wideform.csv', index=False)
# display(tmpdf)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 6 columns):
VP             19 non-null int64
Gender         19 non-null object
age            19 non-null int64
Scale1final    19 non-null float64
Scale2final    19 non-null float64
Scale3final    19 non-null float64
dtypes: float64(3), int64(2), object(1)
memory usage: 992.0+ bytes


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 6 columns):
VP                    19 non-null object
Gender                19 non-null object
Age                   19 non-null int64
Egocentric            19 non-null float64
Allocentric           19 non-null float64
Cardinal Direction    19 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 992.0+ bytes


None

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, 4199 to 7085
Data columns (total 1 columns):
Comments    22 non-null object
dtypes: object(1)
memory usage: 352.0+ bytes


None

,Comments,measurement_1,measurement_2
Subject,,,
4199,third training - #4608 & #3422,4608,3422
8078,third training - #4875 & #4477,4875,4477
8802,third training - #2650 & #9400 firefox map cr...,2650,9400
8021,third training - #6506 & #1385\n,6506,1385
9864,third training - #1216 & #8357,1216,8357
2294,third training - #3599 & #4668,3599,4668
7396,third training - #5046 & #5660,5046,5660
2557,third training - #4684 & #9923,4684,9923
5904,third training - #7429 & #8717,7429,8717


,Subject,Task,Time,Accuracy,Group,measurement_1,measurement_2,VP,Scale,Score
0,1719,Absolute,3s,0.500000,Map,4051,3005,4051,Egocentric,2.50
1,1719,Absolute,3s,0.500000,Map,4051,3005,4051,Allocentric,3.86
2,1719,Absolute,3s,0.500000,Map,4051,3005,4051,Cardinal Direction,1.00
3,1719,Absolute,Inf,0.611111,Map,4051,3005,4051,Egocentric,2.50
4,1719,Absolute,Inf,0.611111,Map,4051,3005,4051,Allocentric,3.86
5,1719,Absolute,Inf,0.611111,Map,4051,3005,4051,Cardinal Direction,1.00
6,1719,Relative,3s,0.500000,Map,4051,3005,4051,Egocentric,2.50
7,1719,Relative,3s,0.500000,Map,4051,3005,4051,Allocentric,3.86
8,1719,Relative,3s,0.500000,Map,4051,3005,4051,Cardinal Direction,1.00
9,1719,Relative,Inf,0.388889,Map,4051,3005,4051,Egocentric,2.50


### Get subject indices and merge with FRS scales to join VR task accuracy and FRS score in one df

In [43]:
FRS_VR = pd.read_csv(DATA_PATH + 'FRS_scales_VR.csv', sep=';', na_values=' ') 
FRS_VR.columns = ['VP', 'Gender','Age', 'Egocentric','Allocentric','Cardinal Direction']
FRS_VR.VP = FRS_VR.VP.astype(str)
FRS_VR_long = pd.melt(FRS_VR, id_vars=['VP'], value_vars=['Egocentric','Allocentric','Cardinal Direction'],
                     var_name='Scale',value_name='Score')

display(FRS_VR.info())
tmpdf = (
    subdf
    .query('Discarded != "yes"')
    .query('Measurement == 3')
    .query('Training == "VR"')
    [['Subject','Comments']]
)
tmpdf.Subject = tmpdf.Subject.astype(str)
display(tmpdf.Subject.nunique())

tmpdf = tmpdf.set_index('Subject')

# display(tmpdf.info())
tmpdf[['measurement_1','measurement_2']] = tmpdf.Comments.str.replace(r'[A-Za-z\s\-\&\.\']','')\
                                                .str.strip()\
                                                .str.extract('\#([0-9]*)\#([0-9]*)', expand = True)
tmpdf.drop(columns='Comments', inplace=True)
tmpdf.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_VR_repeated_measurement_subID.csv')
display(tmpdf.measurement_1.nunique())
VR_perf_df.Subject = VR_perf_df.Subject.astype(str)
VR_measurements = VR_perf_df.merge(tmpdf, on='Subject')
VR_FRS_task_long = VR_measurements.merge(FRS_VR_long,left_on='measurement_1', right_on='VP')
display(VR_FRS_task_long)
VR_FRS_task_long.drop(columns=['VP'], inplace=True)
VR_FRS_task_long.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_VR_task_time_FRS_long.csv', index=False)

pivot_VR = pd.pivot_table(VR_perf_df, 
                   index=['Subject','Group'], 
                   values='Accuracy',
                   columns=['Task','Time'],
                   aggfunc='mean').reset_index()
pivot_VR.Subject = pivot_VR.Subject.astype(str)
pivot_VR.columns = ['_'.join(col).rstrip('_') for col in pivot_VR.columns.values]
tmpdf = tmpdf.merge(pivot_VR, on='Subject')
display(tmpdf.measurement_1.nunique())
tmpdf = tmpdf.merge(FRS_VR, left_on='measurement_1', right_on='VP')
tmpdf.drop(columns=['VP'], inplace=True)
display(tmpdf.measurement_1.nunique())
tmpdf.to_csv(path_or_buf = PROCESSED_DATA_PATH+'seahaven_VR_task_time_FRS_wideform.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 6 columns):
VP                    22 non-null object
Gender                22 non-null object
Age                   22 non-null int64
Egocentric            19 non-null float64
Allocentric           19 non-null float64
Cardinal Direction    19 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.1+ KB


None

22

22

,Subject,Task,Time,Accuracy,Group,measurement_1,measurement_2,VP,Scale,Score
0,1089,Absolute,3s,0.416667,VR,2011,1529,2011,Egocentric,4.70
1,1089,Absolute,3s,0.416667,VR,2011,1529,2011,Allocentric,3.42
2,1089,Absolute,3s,0.416667,VR,2011,1529,2011,Cardinal Direction,1.00
3,1089,Absolute,Inf,0.472222,VR,2011,1529,2011,Egocentric,4.70
4,1089,Absolute,Inf,0.472222,VR,2011,1529,2011,Allocentric,3.42
5,1089,Absolute,Inf,0.472222,VR,2011,1529,2011,Cardinal Direction,1.00
6,1089,Relative,3s,0.555556,VR,2011,1529,2011,Egocentric,4.70
7,1089,Relative,3s,0.555556,VR,2011,1529,2011,Allocentric,3.42
8,1089,Relative,3s,0.555556,VR,2011,1529,2011,Cardinal Direction,1.00
9,1089,Relative,Inf,0.611111,VR,2011,1529,2011,Egocentric,4.70


22

22

## SCRATCH PAD